# 데이터셋 전처리 및 저장 과정

## 데이터 로드 및 준비
우선, CIFAR100 이미지 데이터를 로드하고 준비하는 과정부터 설명하겠습니다. 데이터는 두 가지 경로로 구성되어 있으며, 각각 `train` 폴더와 `test` 폴더에 저장되어 있습니다. 우리는 `glob` 라이브러리를 사용해 해당 경로에 있는 모든 `.png` 파일을 검색합니다.

In [1]:
import glob
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from datasets import Dataset, DatasetDict, ClassLabel, Image

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 상위 폴더로 이동
os.chdir('..')

## dataset

밑에 코드의 경로 부분만 변경하면 custom dataset으로도 얼마든지 훈련과 추론을 해볼 수 있습니다. 본인의 데이터셋으로도 한 번 해보세요!

In [3]:
train_dirs = glob.glob('./data/cifar100_images/train/**/*.png', recursive=True)
test_dirs = glob.glob('./data/cifar100_images/test/**/*.png', recursive=True)

In [4]:
len(train_dirs), len(test_dirs)

(50000, 10000)

## 데이터프레임 생성

다음으로, 각각의 경로에서 파일명을 추출하고 이를 데이터프레임으로 변환하여 label과 name 컬럼을 추가합니다. 이 과정에서는 이미지 경로에서 클래스 라벨을 추출하여 각 이미지를 구분합니다.

In [5]:
train_df = pd.DataFrame({'dirs': train_dirs})
test_df = pd.DataFrame({'dirs': test_dirs})

train_df['label'] = train_df['dirs'].apply(lambda x: x.split('/')[-2])
test_df['label'] = test_df['dirs'].apply(lambda x: x.split('/')[-2])

train_df['name'] = train_df['dirs'].apply(lambda x: x.split('/')[-1].split('.')[0])
test_df['name'] = test_df['dirs'].apply(lambda x: x.split('/')[-1].split('.')[0])

In [6]:
train_df

,dirs,label,name
0,./data/cifar100_images/train/cattle/image_0.png,cattle,image_0
1,./data/cifar100_images/train/cattle/image_137.png,cattle,image_137
2,./data/cifar100_images/train/cattle/image_180.png,cattle,image_180
3,./data/cifar100_images/train/cattle/image_224.png,cattle,image_224
4,./data/cifar100_images/train/cattle/image_278.png,cattle,image_278
...,...,...,...
49995,./data/cifar100_images/train/can/image_49863.png,can,image_49863
49996,./data/cifar100_images/train/can/image_49928.png,can,image_49928
49997,./data/cifar100_images/train/can/image_49953.png,can,image_49953
49998,./data/cifar100_images/train/can/image_49965.png,can,image_49965


## 판다스 데이터프레임에 이미지 데이터를 추가

Pandas DataFrame에 이미지 자체를 직접 저장하는 방식으로는, 이미지 데이터를 바이너리 데이터 (바이트 배열) 형태로 저장할 수 있습니다. 그러나 Pandas 자체로는 이미지를 시각화하거나 바로 처리할 수는 없기 때문에, 주로 바이트 배열 형태로 이미지를 저장한 후 다시 이미지로 변환하는 방식이 일반적입니다.

In [7]:
def image_to_byte(row):
    with open(row, 'rb') as image_file:
        return image_file.read()

In [8]:
train_df['image'] = train_df['dirs'].map(image_to_byte)
test_df['image'] = test_df['dirs'].map(image_to_byte)

## 훈련 및 검증 데이터 분할
다음으로, 훈련 데이터를 80:20 비율로 훈련과 검증 데이터로 나눕니다. 이를 위해 train_test_split 함수를 사용하여 데이터를 분할합니다.

In [9]:
train_df, valid_df= train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=42)

데이터가 잘 나누어졌나 확인해봅시다.

In [10]:
len(train_df), len(valid_df), len(test_df)

(40000, 10000, 10000)

In [11]:
train_df['label'].value_counts()

label
turtle        400
fox           400
rabbit        400
couch         400
crocodile     400
             ... 
kangaroo      400
palm_tree     400
forest        400
baby          400
lawn_mower    400
Name: count, Length: 100, dtype: int64

In [12]:
test_df

,dirs,label,name,image
0,./data/cifar100_images/test/mountain/image_0.png,mountain,image_0,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
1,./data/cifar100_images/test/mountain/image_21.png,mountain,image_21,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
2,./data/cifar100_images/test/mountain/image_42.png,mountain,image_42,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
3,./data/cifar100_images/test/mountain/image_212...,mountain,image_212,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
4,./data/cifar100_images/test/mountain/image_397...,mountain,image_397,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
...,...,...,...,...
9995,./data/cifar100_images/test/tiger/image_8975.png,tiger,image_8975,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
9996,./data/cifar100_images/test/tiger/image_9002.png,tiger,image_9002,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
9997,./data/cifar100_images/test/tiger/image_9079.png,tiger,image_9079,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
9998,./data/cifar100_images/test/tiger/image_9490.png,tiger,image_9490,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...


## dataframe을 Dataset 객체로 변환
huggingface library에서 사용할 수 있도록 dataset 객체로 바꿔줍시다.

In [13]:
# 안 해주면 dataset으로 바꾸는 과정에서 __index_level_0라는 칼럼이 생깁니다. index가 정렬이 안되어있어서 중요한 정보라고 생각되어 자동으로 바뀌는 과정같습니다.
# 이걸 방지해주기 위해서 reset_index를 통해서 다시 정렬해줍니다.
train_df.reset_index(drop=True, inplace=True)
valid_df.reset_index(drop=True, inplace=True)

In [14]:
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)
test_dataset = Dataset.from_pandas(test_df)

여기서 dataset에 저장될 이미지를 사이즈를 resize해줍니다. cifar10의 이미지는 32x32입니다. 현재 사용되는 모델들의 표준 사이즈는 224x224가 표준 사이즈입니다. 사용하기 위해서 256x256으로 resize하고 이후 crop하여 사용할 것입니다.

In [15]:
def convert_size(batch):
    batch['image'] = [img.convert('RGB').resize((256, 256)) for img in batch['image']]
    return batch

In [16]:
def resize_and_cast(dataset, new_features):
    dataset = dataset.cast(new_features)
    dataset = dataset.map(convert_size, batched=True, batch_size=256)
    return dataset

In [17]:
new_features = train_dataset.features.copy()
new_features['label'] = ClassLabel(names=list(set(train_dataset['label'])))
new_features['image'] = Image()

train_dataset = resize_and_cast(train_dataset, new_features)
valid_dataset = resize_and_cast(valid_dataset, new_features)
test_dataset = resize_and_cast(test_dataset, new_features)

Map: 100%|██████████| 10000/10000 [07:11<00:00, 23.17 examples/s]


Map:  84%|████████▍ | 8448/10000 [06:13<01:03, 24.27 examples/s]]

## 최종 데이터 저장
모든 데이터셋을 전처리한 후, 해당 데이터를 디스크에 저장합니다.

In [18]:
combined_dataset = DatasetDict({
        "train": train_dataset,
        "valid": valid_dataset})
test_dataset = DatasetDict(
        {"test" : test_dataset}
)

In [19]:
combined_dataset.save_to_disk('./data/hfdataset/train_valid')
test_dataset.save_to_disk('./data/hfdataset/test')

Saving the dataset (0/4 shards):   0%|          | 0/40000 [00:00<?, ? examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 10000/10000 [00:01<00:00, 8184.15 examples/s]
